# Complete LangChain GraphRAG Pipeline

This notebook demonstrates:
1. Building a complete knowledge graph using the ADK workflow from notebooks 1-8
2. Implementing LangChain GraphRAG for advanced information retrieval
3. Multiple retrieval strategies with Q&A chains

## Use Case: Supply Chain Analysis with Product Reviews

We'll build a knowledge graph that connects:
- **Domain Graph**: Products, Assemblies, Parts, and Suppliers (from CSV files)
- **Subject Graph**: Extracted entities from product reviews (Issues, Features, Locations)
- **Lexical Graph**: Document chunks with embeddings for semantic search

---

## Part 1: Environment Setup

Setting up all necessary imports and connections for both ADK and LangChain.

In [ ]:
# Core imports
import os
import re
import asyncio
from pathlib import Path
from typing import Dict, Any, List, Optional, Tuple
from itertools import islice

# Google ADK imports
from google.adk.agents import Agent
from google.adk.models.lite_llm import LiteLlm

# Neo4j imports
from neo4j import GraphDatabase
from neo4j_graphrag.experimental.pipeline.kg_builder import SimpleKGPipeline
from neo4j_graphrag.experimental.components.text_splitters.base import TextSplitter
from neo4j_graphrag.experimental.components.types import TextChunk, TextChunks, PdfDocument, DocumentInfo
from neo4j_graphrag.experimental.components.pdf_loader import DataLoader
from neo4j_graphrag.llm import OpenAILLM
from neo4j_graphrag.embeddings import OpenAIEmbeddings as Neo4jOpenAIEmbeddings

# LangChain imports
from langchain_neo4j import Neo4jGraph, Neo4jVector
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.chains import GraphCypherQAChain
from langchain.prompts import PromptTemplate
from langchain_core.documents import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter

# LangGraph imports for advanced workflows
from langgraph.graph import StateGraph, END
from typing import TypedDict

# Custom workshop modules
from neo4j_for_adk import graphdb, tool_success, tool_error
from helper import get_neo4j_import_dir
from tools import drop_neo4j_indexes, clear_neo4j_data

# Disable warnings for clean output
import warnings
warnings.filterwarnings('ignore')

import logging
logging.basicConfig(level=logging.CRITICAL)

print("✅ All libraries imported successfully")

In [ ]:
# Load environment variables
from dotenv import load_dotenv
load_dotenv()

# Get configuration from environment
NEO4J_URI = os.getenv("NEO4J_URI", "bolt://localhost:7687")
NEO4J_USERNAME = os.getenv("NEO4J_USERNAME", "neo4j")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
NEO4J_IMPORT_DIR = get_neo4j_import_dir()

# Verify connections
print(f"Neo4j URI: {NEO4J_URI}")
print(f"Neo4j Username: {NEO4J_USERNAME}")
print(f"Neo4j Import Dir: {NEO4J_IMPORT_DIR}")
print(f"OpenAI API Key: {'✅ Set' if OPENAI_API_KEY else '❌ Missing'}")

# Test Neo4j connection
neo4j_test = graphdb.send_query("RETURN 'Neo4j is Ready!' as message")
print(f"\nNeo4j Connection: {neo4j_test}")

In [ ]:
# Initialize LLM clients
MODEL_GPT_4O = "openai/gpt-4o"

# ADK LLM
llm_adk = LiteLlm(model=MODEL_GPT_4O)

# LangChain LLM
llm_langchain = ChatOpenAI(model="gpt-4o", temperature=0)

# Neo4j GraphRAG LLM
llm_neo4j = OpenAILLM(model_name="gpt-4o", model_params={"temperature": 0})

# Embeddings
embedder_langchain = OpenAIEmbeddings(model="text-embedding-3-large")
embedder_neo4j = Neo4jOpenAIEmbeddings(model="text-embedding-3-large")

print("✅ LLM clients initialized")

---

## Part 2: ADK-Based Knowledge Graph Construction

We'll follow the exact workflow from the notebooks but programmatically:
1. Define user intent (supply chain analysis)
2. Select files for import
3. Design schema
4. Construct domain graph from CSVs
5. Construct subject graph from markdown reviews

In [ ]:
# Clear existing graph data (optional - be careful!)
print("Clearing existing graph data...")
drop_neo4j_indexes()
clear_neo4j_data()
print("✅ Graph cleared")

### 2.1 Define User Intent and Files

In [ ]:
# Define the approved user goal (from Notebook 2)
approved_user_goal = {
    "kind_of_graph": "supply chain analysis",
    "description": """A multi-level bill of materials for manufactured products, 
    useful for root cause analysis. Includes product reviews to trace quality 
    issues back to suppliers and components."""
}

# Define approved files for import (from Notebook 3)
approved_csv_files = [
    'products.csv',
    'assemblies.csv', 
    'parts.csv',
    'part_supplier_mapping.csv',
    'suppliers.csv'
]

approved_markdown_files = [
    "product_reviews/gothenburg_table_reviews.md",
    "product_reviews/helsingborg_dresser_reviews.md",
    "product_reviews/jonkoping_coffee_table_reviews.md",
    "product_reviews/linkoping_bed_reviews.md",
    "product_reviews/malmo_desk_reviews.md",
    "product_reviews/norrkoping_nightstand_reviews.md",
    "product_reviews/orebro_lamp_reviews.md",
    "product_reviews/stockholm_chair_reviews.md",
    "product_reviews/uppsala_sofa_reviews.md",
    "product_reviews/vasteras_bookshelf_reviews.md"
]

print(f"User Goal: {approved_user_goal['kind_of_graph']}")
print(f"CSV Files: {len(approved_csv_files)}")
print(f"Markdown Files: {len(approved_markdown_files)}")

### 2.2 Define Schema Construction Plan

In [ ]:
# Define the approved construction plan (from Notebook 4)
approved_construction_plan = {
    "Assembly": {
        "construction_type": "node", 
        "source_file": "assemblies.csv", 
        "label": "Assembly", 
        "unique_column_name": "assembly_id", 
        "properties": ["assembly_name", "quantity", "product_id"]
    }, 
    "Part": {
        "construction_type": "node", 
        "source_file": "parts.csv", 
        "label": "Part", 
        "unique_column_name": "part_id", 
        "properties": ["part_name", "quantity", "assembly_id"]
    }, 
    "Product": {
        "construction_type": "node", 
        "source_file": "products.csv", 
        "label": "Product", 
        "unique_column_name": "product_id", 
        "properties": ["product_name", "price", "description"]
    }, 
    "Supplier": {
        "construction_type": "node", 
        "source_file": "suppliers.csv", 
        "label": "Supplier", 
        "unique_column_name": "supplier_id", 
        "properties": ["name", "specialty", "city", "country", "website", "contact_email"]
    }, 
    "Contains": {
        "construction_type": "relationship", 
        "source_file": "assemblies.csv", 
        "relationship_type": "Contains", 
        "from_node_label": "Product", 
        "from_node_column": "product_id", 
        "to_node_label": "Assembly", 
        "to_node_column": "assembly_id", 
        "properties": ["quantity"]
    }, 
    "Is_Part_Of": {
        "construction_type": "relationship", 
        "source_file": "parts.csv", 
        "relationship_type": "Is_Part_Of", 
        "from_node_label": "Part", 
        "from_node_column": "part_id", 
        "to_node_label": "Assembly", 
        "to_node_column": "assembly_id", 
        "properties": ["quantity"]
    }, 
    "Supplied_By": {
        "construction_type": "relationship", 
        "source_file": "part_supplier_mapping.csv", 
        "relationship_type": "Supplied_By", 
        "from_node_label": "Part", 
        "from_node_column": "part_id", 
        "to_node_label": "Supplier", 
        "to_node_column": "supplier_id", 
        "properties": ["supplier_name", "lead_time_days", "unit_cost", "minimum_order_quantity", "preferred_supplier"]
    }
}

print("Construction Plan:")
nodes = [k for k, v in approved_construction_plan.items() if v['construction_type'] == 'node']
relationships = [k for k, v in approved_construction_plan.items() if v['construction_type'] == 'relationship']
print(f"  Nodes: {nodes}")
print(f"  Relationships: {relationships}")

### 2.3 Construct Domain Graph from CSVs

In [ ]:
def create_uniqueness_constraint(label: str, unique_property_key: str) -> Dict[str, Any]:
    """Creates a uniqueness constraint for a node label and property key."""
    constraint_name = f"{label}_{unique_property_key}_constraint"
    query = f"""CREATE CONSTRAINT `{constraint_name}` IF NOT EXISTS
    FOR (n:`{label}`)
    REQUIRE n.`{unique_property_key}` IS UNIQUE"""
    return graphdb.send_query(query)

def load_nodes_from_csv(source_file: str, label: str, unique_column_name: str, properties: list[str]) -> Dict[str, Any]:
    """Load nodes from CSV file."""
    query = f"""LOAD CSV WITH HEADERS FROM "file:///" + $source_file AS row
    CALL (row) {{
        MERGE (n:$($label) {{ {unique_column_name} : row[$unique_column_name] }})
        FOREACH (k IN $properties | SET n[k] = row[k])
    }} IN TRANSACTIONS OF 1000 ROWS
    """
    return graphdb.send_query(query, {
        "source_file": source_file,
        "label": label,
        "unique_column_name": unique_column_name,
        "properties": properties
    })

def import_nodes(node_construction: dict) -> dict:
    """Import nodes as defined by a node construction rule."""
    # Create uniqueness constraint
    uniqueness_result = create_uniqueness_constraint(
        node_construction["label"],
        node_construction["unique_column_name"]
    )
    if uniqueness_result["status"] == "error":
        return uniqueness_result
    
    # Import nodes from csv
    return load_nodes_from_csv(
        node_construction["source_file"],
        node_construction["label"],
        node_construction["unique_column_name"],
        node_construction["properties"]
    )

def import_relationships(relationship_construction: dict) -> Dict[str, Any]:
    """Import relationships as defined by a relationship construction rule."""
    from_node_column = relationship_construction["from_node_column"]
    to_node_column = relationship_construction["to_node_column"]
    
    query = f"""LOAD CSV WITH HEADERS FROM "file:///" + $source_file AS row
    CALL (row) {{
        MATCH (from_node:$($from_node_label) {{ {from_node_column} : row[$from_node_column] }}),
              (to_node:$($to_node_label) {{ {to_node_column} : row[$to_node_column] }} )
        MERGE (from_node)-[r:$($relationship_type)]->(to_node)
        FOREACH (k IN $properties | SET r[k] = row[k])
    }} IN TRANSACTIONS OF 1000 ROWS
    """
    
    return graphdb.send_query(query, {
        "source_file": relationship_construction["source_file"],
        "from_node_label": relationship_construction["from_node_label"],
        "from_node_column": relationship_construction["from_node_column"],
        "to_node_label": relationship_construction["to_node_label"],
        "to_node_column": relationship_construction["to_node_column"],
        "relationship_type": relationship_construction["relationship_type"],
        "properties": relationship_construction["properties"]
    })

In [ ]:
def construct_domain_graph(construction_plan: dict) -> None:
    """Construct the complete domain graph from CSVs."""
    print("Constructing Domain Graph...")
    
    # First, import nodes
    node_constructions = [value for value in construction_plan.values() if value['construction_type'] == 'node']
    for node_construction in node_constructions:
        print(f"  Importing {node_construction['label']} nodes...")
        result = import_nodes(node_construction)
        if result['status'] == 'error':
            print(f"    ❌ Error: {result.get('error_message', 'Unknown error')}")
        else:
            print(f"    ✅ Success")
    
    # Second, import relationships
    relationship_constructions = [value for value in construction_plan.values() if value['construction_type'] == 'relationship']
    for relationship_construction in relationship_constructions:
        print(f"  Creating {relationship_construction['relationship_type']} relationships...")
        result = import_relationships(relationship_construction)
        if result['status'] == 'error':
            print(f"    ❌ Error: {result.get('error_message', 'Unknown error')}")
        else:
            print(f"    ✅ Success")

# Construct the domain graph
construct_domain_graph(approved_construction_plan)

# Verify the domain graph
stats = graphdb.send_query("""
MATCH (n)
WITH labels(n) as labels, count(n) as count
RETURN labels[0] as label, count
ORDER BY label
""")

print("\nDomain Graph Statistics:")
for row in stats['query_result']:
    print(f"  {row['label']}: {row['count']} nodes")

### 2.4 Define Entities and Facts for Extraction from Reviews

In [ ]:
# Define approved entities (from Notebook 5)
approved_entities = ['Product', 'Issue', 'Feature', 'Location']

# Define approved fact types (from Notebook 5)
approved_fact_types = {
    'has_issue': {
        'subject_label': 'Product',
        'predicate_label': 'has_issue',
        'object_label': 'Issue'
    },
    'includes_feature': {
        'subject_label': 'Product',
        'predicate_label': 'includes_feature',
        'object_label': 'Feature'
    },
    'used_in_location': {
        'subject_label': 'Product',
        'predicate_label': 'used_in_location',
        'object_label': 'Location'
    }
}

print("Entity Types:", approved_entities)
print("\nFact Types:")
for key, fact in approved_fact_types.items():
    print(f"  {fact['subject_label']} --{fact['predicate_label'].upper()}--> {fact['object_label']}")

### 2.5 Construct Subject Graph from Markdown Reviews

In [ ]:
# Custom text splitter for markdown
class RegexTextSplitter(TextSplitter):
    """Split text using regex matched delimiters."""
    def __init__(self, re_pattern: str):
        self.re_pattern = re_pattern
    
    async def run(self, text: str) -> TextChunks:
        texts = re.split(self.re_pattern, text)
        chunks = [TextChunk(text=str(text), index=i) for (i, text) in enumerate(texts)]
        return TextChunks(chunks=chunks)

# Custom markdown loader
class MarkdownDataLoader(DataLoader):
    def extract_title(self, markdown_text):
        pattern = r'^# (.+)$'
        match = re.search(pattern, markdown_text, re.MULTILINE)
        return match.group(1) if match else "Untitled"
    
    async def run(self, filepath: Path, metadata = {}) -> PdfDocument:
        with open(filepath, "r") as f:
            markdown_text = f.read()
        doc_headline = self.extract_title(markdown_text)
        markdown_info = DocumentInfo(
            path=str(filepath),
            metadata={"title": doc_headline}
        )
        return PdfDocument(text=markdown_text, document_info=markdown_info)

print("✅ Custom loaders defined")

In [ ]:
def file_context(file_path: str, num_lines: int = 5) -> str:
    """Extract first few lines of a file for context."""
    with open(file_path, 'r') as f:
        lines = []
        for _ in range(num_lines):
            line = f.readline()
            if not line:
                break
            lines.append(line)
    return "\n".join(lines)

def contextualize_er_extraction_prompt(context: str) -> str:
    """Create entity extraction prompt with context."""
    general_instructions = """
    You are a top-tier algorithm designed for extracting
    information in structured formats to build a knowledge graph.

    Extract the entities (nodes) and specify their type from the following text.
    Also extract the relationships between these nodes.

    Return result as JSON using the following format:
    {{"nodes": [ {{"id": "0", "label": "Person", "properties": {{"name": "John"}} }}],
    "relationships": [{{"type": "KNOWS", "start_node_id": "0", "end_node_id": "1", "properties": {{"since": "2024-08-01"}} }}] }}

    Use only the following node and relationship types (if provided):
    {schema}

    Assign a unique ID (string) to each node, and reuse it to define relationships.
    Do respect the source and target node types for relationship and
    the relationship direction.

    Make sure you adhere to the following rules to produce valid JSON objects:
    - Do not return any additional information other than the JSON in it.
    - Omit any backticks around the JSON - simply output the JSON on its own.
    - The JSON object must not wrapped into a list - it is its own JSON object.
    - Property names must be enclosed in double quotes
    """
    
    context_section = f"""
    Consider the following context to help identify entities and relationships:
    <context>
    {context}  
    </context>"""
    
    input_section = """
    Input text:
    {text}
    """
    
    return general_instructions + "\n" + context_section + "\n" + input_section

print("✅ Extraction prompts defined")

In [ ]:
def make_kg_builder(file_path: str) -> SimpleKGPipeline:
    """Build a KG builder pipeline for a given markdown file."""
    # Get entity schema from approved entities and facts
    schema_node_types = approved_entities
    schema_relationship_types = [key.upper() for key in approved_fact_types.keys()]
    schema_patterns = [
        [fact['subject_label'], fact['predicate_label'].upper(), fact['object_label']]
        for fact in approved_fact_types.values()
    ]
    
    entity_schema = {
        "node_types": schema_node_types,
        "relationship_types": schema_relationship_types,
        "patterns": schema_patterns,
        "additional_node_types": False
    }
    
    # Create contextualized prompt
    context = file_context(file_path)
    contextualized_prompt = contextualize_er_extraction_prompt(context)
    
    # Build pipeline
    return SimpleKGPipeline(
        llm=llm_neo4j,
        driver=graphdb.get_driver(),
        embedder=embedder_neo4j,
        from_pdf=True,
        pdf_loader=MarkdownDataLoader(),
        text_splitter=RegexTextSplitter("---"),
        schema=entity_schema,
        prompt_template=contextualized_prompt,
    )

print("✅ KG builder function defined")

In [ ]:
# Process all markdown files to create subject graph
print("Constructing Subject Graph from Reviews...")

for file_name in approved_markdown_files[:3]:  # Process first 3 files for demo
    file_path = os.path.join(NEO4J_IMPORT_DIR, file_name)
    print(f"  Processing: {file_name}")
    
    try:
        kg_builder = make_kg_builder(file_path)
        results = await kg_builder.run_async(file_path=str(file_path))
        print(f"    ✅ Processed successfully")
    except Exception as e:
        print(f"    ❌ Error: {str(e)}")

print("\n✅ Subject graph construction complete")

### 2.6 Entity Resolution: Connect Subject and Domain Graphs

In [ ]:
def correlate_subject_and_domain_nodes(label: str, entity_key: str, domain_key: str, similarity: float = 0.9) -> dict:
    """Correlate entity and domain nodes based on similarity."""
    results = graphdb.send_query("""
    MATCH (entity:$($entityLabel):`__Entity__`),(domain:$($entityLabel))
    WHERE apoc.text.jaroWinklerDistance(entity[$entityKey], domain[$domainKey]) < $distance
    MERGE (entity)-[r:CORRESPONDS_TO]->(domain)
    ON CREATE SET r.created_at = datetime()
    ON MATCH SET r.updated_at = datetime()
    RETURN $entityLabel as entityLabel, count(r) as relationshipCount
    """, {
        "entityLabel": label,
        "entityKey": entity_key,
        "domainKey": domain_key,
        "distance": (1.0 - similarity)
    })
    return results

# Connect Product entities to Product domain nodes
print("Performing Entity Resolution...")
result = correlate_subject_and_domain_nodes("Product", "name", "product_name", similarity=0.8)
if result['status'] == 'success':
    count = result['query_result'][0]['relationshipCount'] if result['query_result'] else 0
    print(f"  ✅ Connected {count} Product entities to domain nodes")
else:
    print(f"  ❌ Error: {result.get('error_message', 'Unknown')}")

---

## Part 3: LangChain GraphRAG Implementation

Now we'll implement multiple retrieval strategies using LangChain.

### 3.1 Initialize LangChain Neo4j Components

In [ ]:
# Initialize Neo4j Graph for LangChain
graph = Neo4jGraph(
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
    database="neo4j"
)

# Refresh schema
graph.refresh_schema()

print("Graph Schema:")
print(f"  Node Labels: {graph.structured_schema['node_props'].keys()}")
print(f"  Relationships: {graph.structured_schema['relationships']}")

### 3.2 Create Hybrid Vector Store

In [ ]:
# Create vector index on chunks
create_vector_index = graphdb.send_query("""
CREATE VECTOR INDEX `chunk_embedding_index` IF NOT EXISTS
FOR (c:Chunk)
ON (c.embedding)
OPTIONS {
    indexConfig: {
        `vector.dimensions`: 3072,
        `vector.similarity_function`: 'cosine'
    }
}
""")

print("Vector index creation:", create_vector_index['status'])

In [ ]:
# Create full-text index for hybrid search
create_fulltext_index = graphdb.send_query("""
CREATE FULLTEXT INDEX `chunk_text_index` IF NOT EXISTS
FOR (c:Chunk)
ON EACH [c.text]
""")

print("Full-text index creation:", create_fulltext_index['status'])

In [ ]:
# Custom retrieval query that traverses the graph for context
retrieval_query = """
// node is the chunk found by vector/hybrid search
// score is the similarity score

// Get parent document for broader context
OPTIONAL MATCH (doc:Document)-[:HAS_CHUNK]->(node)

// Get entities mentioned in this chunk
OPTIONAL MATCH (entity:`__Entity__`)-[:MENTIONED_IN]->(node)

// Get corresponding domain nodes
OPTIONAL MATCH (entity)-[:CORRESPONDS_TO]->(product:Product)

// Get supply chain info for products
OPTIONAL MATCH (product)-[:Contains]->(assembly:Assembly)
OPTIONAL MATCH (assembly)<-[:Is_Part_Of]-(part:Part)
OPTIONAL MATCH (part)-[:Supplied_By]->(supplier:Supplier)

WITH node, score, doc, 
     collect(DISTINCT entity.name) AS entities,
     collect(DISTINCT product.product_name) AS products,
     collect(DISTINCT supplier.name)[..3] AS suppliers

RETURN 
    node.text AS text,
    score,
    doc.path AS source_document,
    entities,
    products,
    suppliers
"""

print("✅ Custom retrieval query defined")

In [ ]:
# Initialize Hybrid Vector Store with custom retrieval
vector_store = Neo4jVector.from_existing_index(
    embedding=embedder_langchain,
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
    index_name="chunk_embedding_index",
    node_label="Chunk",
    text_node_property="text",
    embedding_node_property="embedding",
    search_type="hybrid",
    retrieval_query=retrieval_query
)

print("✅ Hybrid vector store initialized")

### 3.3 Implement Multiple Retrieval Strategies

In [ ]:
class SupplyChainRAG:
    """Multi-strategy retrieval for supply chain Q&A."""
    
    def __init__(self, vector_store, graph, llm):
        self.vector_store = vector_store
        self.graph = graph
        self.llm = llm
        self.cypher_chain = self._create_cypher_chain()
    
    def _create_cypher_chain(self):
        """Create GraphCypherQAChain with custom prompt."""
        cypher_prompt_template = """
        You are an expert at converting questions about supply chain and product quality
        into Neo4j Cypher queries.
        
        The graph contains:
        - Product nodes with properties: product_id, product_name, price, description
        - Assembly nodes with properties: assembly_id, assembly_name, quantity
        - Part nodes with properties: part_id, part_name, quantity
        - Supplier nodes with properties: supplier_id, name, city, country, specialty
        - Relationships: (Product)-[:Contains]->(Assembly), (Part)-[:Is_Part_Of]->(Assembly), (Part)-[:Supplied_By]->(Supplier)
        - Entity nodes from reviews with labels: Issue, Feature, Location
        
        Schema: {schema}
        
        Question: {question}
        
        Return a valid Cypher query that answers the question.
        """
        
        cypher_prompt = PromptTemplate(
            template=cypher_prompt_template,
            input_variables=["schema", "question"]
        )
        
        return GraphCypherQAChain.from_llm(
            llm=self.llm,
            graph=self.graph,
            cypher_prompt=cypher_prompt,
            verbose=True,
            validate_cypher=True,
            top_k=10
        )
    
    def hybrid_search(self, question: str, k: int = 5) -> List[Document]:
        """Strategy 1: Hybrid vector + full-text search with graph traversal."""
        return self.vector_store.similarity_search(question, k=k)
    
    def cypher_query(self, question: str) -> str:
        """Strategy 2: Direct Cypher query generation."""
        try:
            result = self.cypher_chain.invoke({"query": question})
            return result["result"]
        except Exception as e:
            return f"Error executing Cypher query: {str(e)}"
    
    def trace_issue_to_supplier(self, product_name: str, issue: str) -> str:
        """Strategy 3: Specialized query for root cause analysis."""
        cypher = """
        MATCH (p:Product {product_name: $product_name})
        MATCH (p)-[:Contains]->(a:Assembly)
        MATCH (part:Part)-[:Is_Part_Of]->(a)
        MATCH (part)-[:Supplied_By]->(s:Supplier)
        
        OPTIONAL MATCH (issue:Issue)
        WHERE issue.name CONTAINS $issue
        
        RETURN DISTINCT 
            p.product_name AS product,
            a.assembly_name AS assembly,
            collect(DISTINCT part.part_name)[..5] AS parts,
            collect(DISTINCT s.name)[..5] AS suppliers,
            collect(DISTINCT s.city + ', ' + s.country)[..5] AS locations
        """
        
        result = self.graph.query(cypher, {
            "product_name": product_name,
            "issue": issue
        })
        
        return self._format_trace_results(result)
    
    def _format_trace_results(self, results):
        """Format tracing results into readable text."""
        if not results:
            return "No supply chain information found."
        
        output = []
        for r in results:
            output.append(f"Product: {r['product']}")
            output.append(f"Assembly: {r['assembly']}")
            output.append(f"Parts: {', '.join(r['parts'])}")
            output.append(f"Suppliers: {', '.join(r['suppliers'])}")
            output.append(f"Locations: {', '.join(r['locations'])}")
            output.append("---")
        
        return "\n".join(output)
    
    def answer_question(self, question: str) -> str:
        """Combined strategy: Use multiple retrieval methods and synthesize answer."""
        # Get context from hybrid search
        docs = self.hybrid_search(question, k=3)
        vector_context = "\n\n".join([doc.page_content for doc in docs])
        
        # Try Cypher query for structured data
        cypher_result = self.cypher_query(question)
        
        # Generate final answer
        prompt = f"""
        Answer the following question using the provided context.
        
        Context from reviews and documentation:
        {vector_context}
        
        Context from structured data:
        {cypher_result}
        
        Question: {question}
        
        Provide a comprehensive answer that combines insights from both sources.
        If there are quality issues mentioned, trace them to potential root causes.
        """
        
        response = self.llm.invoke(prompt)
        return response.content

# Initialize the RAG system
rag_system = SupplyChainRAG(vector_store, graph, llm_langchain)
print("✅ Supply Chain RAG system initialized")

---

## Part 4: Advanced Features with LangGraph

In [ ]:
# Define the state for the workflow
class RAGState(TypedDict):
    question: str
    query_type: str
    vector_results: List[str]
    cypher_results: str
    final_context: str
    answer: str

# Workflow functions
def classify_query(state: RAGState) -> RAGState:
    """Classify the query type."""
    question = state["question"]
    
    # Simple classification logic
    if any(word in question.lower() for word in ["supplier", "part", "assembly"]):
        state["query_type"] = "structured"
    elif any(word in question.lower() for word in ["issue", "problem", "quality", "review"]):
        state["query_type"] = "unstructured"
    else:
        state["query_type"] = "both"
    
    return state

def vector_search(state: RAGState) -> RAGState:
    """Perform vector search."""
    docs = rag_system.hybrid_search(state["question"], k=3)
    state["vector_results"] = [doc.page_content for doc in docs]
    return state

def cypher_search(state: RAGState) -> RAGState:
    """Execute Cypher search."""
    state["cypher_results"] = rag_system.cypher_query(state["question"])
    return state

def aggregate_results(state: RAGState) -> RAGState:
    """Combine results from multiple sources."""
    context_parts = []
    
    if state.get("vector_results"):
        context_parts.append("From reviews and documentation:")
        context_parts.extend(state["vector_results"])
    
    if state.get("cypher_results"):
        context_parts.append("\nFrom structured data:")
        context_parts.append(state["cypher_results"])
    
    state["final_context"] = "\n\n".join(context_parts)
    return state

def generate_answer(state: RAGState) -> RAGState:
    """Generate final answer."""
    prompt = f"""
    Answer this question based on the context:
    
    Context:
    {state['final_context']}
    
    Question: {state['question']}
    """
    
    response = llm_langchain.invoke(prompt)
    state["answer"] = response.content
    return state

# Build the workflow
workflow = StateGraph(RAGState)

# Add nodes
workflow.add_node("classify", classify_query)
workflow.add_node("vector", vector_search)
workflow.add_node("cypher", cypher_search)
workflow.add_node("aggregate", aggregate_results)
workflow.add_node("generate", generate_answer)

# Add edges with conditional routing
workflow.set_entry_point("classify")

def route_after_classification(state):
    if state["query_type"] == "structured":
        return "cypher"
    elif state["query_type"] == "unstructured":
        return "vector"
    else:
        return "vector"  # Start with vector for "both"

workflow.add_conditional_edges(
    "classify",
    route_after_classification,
    {
        "vector": "vector",
        "cypher": "cypher"
    }
)

# Connect the rest of the workflow
workflow.add_edge("vector", "cypher")  # After vector, also do cypher
workflow.add_edge("cypher", "aggregate")
workflow.add_edge("aggregate", "generate")
workflow.add_edge("generate", END)

# Compile the workflow
app = workflow.compile()

print("✅ LangGraph workflow compiled")

---

## Part 5: Interactive Demo

Test the system with various queries to demonstrate different retrieval strategies.

In [ ]:
# Test queries demonstrating different capabilities
test_queries = [
    "Which suppliers are responsible for quality issues in the Uppsala Sofa?",
    "What are the most common problems across all furniture products?",
    "Find all products that use parts from suppliers in Sweden",
    "Which assembly has the most customer complaints?",
    "What quality issues are mentioned in the Stockholm Chair reviews?",
    "List all suppliers and their specialties",
    "How many parts does the Gothenburg Table have?"
]

print("Test Queries:")
for i, q in enumerate(test_queries, 1):
    print(f"{i}. {q}")

In [ ]:
def test_retrieval_strategy(question: str, strategy: str = "all"):
    """Test different retrieval strategies."""
    print(f"\n{'='*80}")
    print(f"Question: {question}")
    print(f"Strategy: {strategy}")
    print(f"{'='*80}")
    
    if strategy == "hybrid" or strategy == "all":
        print("\n--- Hybrid Search Results ---")
        docs = rag_system.hybrid_search(question, k=2)
        for i, doc in enumerate(docs, 1):
            print(f"\nResult {i}:")
            print(f"  Text: {doc.page_content[:200]}...")
            if doc.metadata:
                print(f"  Metadata: {doc.metadata}")
    
    if strategy == "cypher" or strategy == "all":
        print("\n--- Cypher Query Results ---")
        result = rag_system.cypher_query(question)
        print(result)
    
    if strategy == "combined" or strategy == "all":
        print("\n--- Combined Answer ---")
        answer = rag_system.answer_question(question)
        print(answer)
    
    if strategy == "workflow":
        print("\n--- LangGraph Workflow ---")
        result = app.invoke({"question": question})
        print(f"Query Type: {result['query_type']}")
        print(f"\nAnswer: {result['answer']}")

In [ ]:
# Test first query with all strategies
test_retrieval_strategy(test_queries[0], "all")

In [ ]:
# Test workflow on a complex query
test_retrieval_strategy(test_queries[1], "workflow")

### Interactive Query Interface

In [ ]:
def interactive_qa():
    """Interactive Q&A interface."""
    print("\n" + "="*80)
    print("Supply Chain Knowledge Graph Q&A System")
    print("="*80)
    print("\nType 'quit' to exit, 'help' for example queries\n")
    
    while True:
        question = input("\nYour question: ").strip()
        
        if question.lower() == 'quit':
            print("Goodbye!")
            break
        
        if question.lower() == 'help':
            print("\nExample queries:")
            for q in test_queries[:3]:
                print(f"  - {q}")
            continue
        
        if not question:
            continue
        
        try:
            # Use the workflow for comprehensive answers
            print("\nProcessing...")
            result = app.invoke({"question": question})
            
            print("\n" + "-"*40)
            print("Answer:")
            print("-"*40)
            print(result['answer'])
            
        except Exception as e:
            print(f"\nError: {str(e)}")
            print("Falling back to simple search...")
            answer = rag_system.answer_question(question)
            print(answer)

# Run interactive interface
# interactive_qa()  # Uncomment to run

### Graph Visualization

In [ ]:
def visualize_subgraph(product_name: str):
    """Generate Cypher for visualizing a product's supply chain."""
    cypher = f"""
    // Visualization query for {product_name}
    MATCH path = (p:Product {{product_name: '{product_name}'}})-[:Contains]->(a:Assembly)
    OPTIONAL MATCH parts_path = (part:Part)-[:Is_Part_Of]->(a)
    OPTIONAL MATCH supplier_path = (part)-[:Supplied_By]->(s:Supplier)
    OPTIONAL MATCH entity_path = (e:`__Entity__`:Product)-[:CORRESPONDS_TO]->(p)
    OPTIONAL MATCH issue_path = (e)-[:HAS_ISSUE]->(issue:Issue)
    
    RETURN path, parts_path, supplier_path, entity_path, issue_path
    LIMIT 50
    """
    
    print(f"To visualize {product_name} supply chain in Neo4j Browser, run:")
    print("\n" + cypher)
    
    # Get statistics
    stats_cypher = f"""
    MATCH (p:Product {{product_name: '{product_name}'}})
    OPTIONAL MATCH (p)-[:Contains]->(a:Assembly)
    OPTIONAL MATCH (part:Part)-[:Is_Part_Of]->(a)
    OPTIONAL MATCH (part)-[:Supplied_By]->(s:Supplier)
    RETURN 
        count(DISTINCT a) as assemblies,
        count(DISTINCT part) as parts,
        count(DISTINCT s) as suppliers
    """
    
    stats = graph.query(stats_cypher)
    if stats:
        print(f"\nSupply Chain Statistics for {product_name}:")
        print(f"  Assemblies: {stats[0]['assemblies']}")
        print(f"  Parts: {stats[0]['parts']}")
        print(f"  Suppliers: {stats[0]['suppliers']}")

# Example visualization
visualize_subgraph("Uppsala Sofa")

---

## Summary and Evaluation

### What We've Built:

1. **Complete Knowledge Graph**:
   - Domain graph from structured CSV data
   - Subject graph from unstructured markdown reviews
   - Lexical graph with embeddings for semantic search
   - Entity resolution linking graphs together

2. **Multiple Retrieval Strategies**:
   - Hybrid search (vector + full-text) with graph traversal
   - Cypher generation for structured queries
   - Specialized tracing for root cause analysis
   - LangGraph workflow for intelligent routing

3. **Supply Chain Q&A Capabilities**:
   - Trace quality issues to suppliers
   - Analyze patterns across products
   - Combine structured and unstructured data
   - Provide context-rich answers

### Performance Metrics

In [ ]:
# Get final graph statistics
final_stats = graphdb.send_query("""
MATCH (n)
WITH labels(n) as node_labels, count(n) as count
UNWIND node_labels as label
WITH label, sum(count) as total
RETURN label, total
ORDER BY total DESC
""")

print("Final Knowledge Graph Statistics:")
print("-" * 40)
total_nodes = 0
for row in final_stats['query_result']:
    print(f"{row['label']:20} {row['total']:10,} nodes")
    total_nodes += row['total']

print("-" * 40)
print(f"{'Total':20} {total_nodes:10,} nodes")

# Count relationships
rel_stats = graphdb.send_query("""
MATCH ()-[r]->()
RETURN type(r) as type, count(r) as count
ORDER BY count DESC
""")

print("\nRelationship Statistics:")
print("-" * 40)
total_rels = 0
for row in rel_stats['query_result']:
    print(f"{row['type']:20} {row['count']:10,} relationships")
    total_rels += row['count']

print("-" * 40)
print(f"{'Total':20} {total_rels:10,} relationships")

## Conclusion

This notebook demonstrated:

1. **Complete pipeline** from raw data to working Q&A system
2. **Integration** of ADK workflow with LangChain GraphRAG
3. **Multiple retrieval strategies** for different query types
4. **Real-world use case** with supply chain analysis

The system can now:
- Answer complex supply chain questions
- Trace quality issues to root causes
- Combine insights from reviews and structured data
- Provide context-aware, accurate responses

### Next Steps:

1. Process all review files (we only did 3 for demo)
2. Add more sophisticated entity resolution
3. Implement evaluation metrics
4. Deploy as API service
5. Add caching for performance
6. Create dashboard for visualization